In [ ]:
%matplotlib inline

In [1]:
print(__doc__)
import numpy as np
import json
from sklearn.datasets import load_boston
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from skopt import gp_minimize
import pickle
import xgboost as xgb
from xgboost import XGBRegressor
from skopt.space import Real, Integer
from skopt.utils import use_named_args

Automatically created module for IPython interactive environment


In [3]:
BIDS_path = 'C:\\Users\\ICN_admin\\Documents\\BIDS\\'
subject_id = ['000', '001', '004', '005', '006', '007', '008', '009', '010', '013', '014']
out_path = out_path = 'C:\\Users\\ICN_admin\\Documents\\ECOGVSTN\\dat\\'

In [4]:
def append_time_dim(arr, y_, time_stamps):
    """
    apply added time dimension for the data array and label given time_stamps
    """
    time_arr = np.zeros([arr.shape[0]-time_stamps, int(time_stamps*arr.shape[1])])
    for time_idx, time_ in enumerate(np.arange(time_stamps, arr.shape[0])):
        for time_point in range(time_stamps):
            time_arr[time_idx, time_point*arr.shape[1]:(time_point+1)*arr.shape[1]] = arr[time_-time_point,:]
    return time_arr, y_[time_stamps:]

In [5]:
# The list of hyper-parameters we want to optimize. For each one we define the
# bounds, the corresponding scikit-learn parameter name, as well as how to
# sample values from that dimension (`'log-uniform'` for the learning rate)
n_features = 40
space_XGB  = [Integer(1, 100, name='max_depth'),
          Real(10**-5, 10**0, "log-uniform", name='learning_rate'),
          Integer(10**0, 10**3, "log-uniform", name='n_estimators'), 
          Real(10**0, 10**1, "uniform", name="gamma")]

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    # return a pair metric_name, result. The metric name must not contain a
    # colon (:) or a space since preds are margin(before logistic
    # transformation, cutoff at 0)
    
    r2 = metrics.r2_score(labels, preds)
    
    if r2 < 0:
        r2 = 0
    
    return 'r2', r2

# this decorator allows your objective function to receive a the parameters as
# keyword arguments. This is particularly convenient when you want to set
# scikit-learn estimator parameters
@use_named_args(space_XGB)
def objective_XGB(**params):
    print(params)
    
    params_ = {'max_depth': int(params["max_depth"]),
         'gamma': params['gamma'],
         'n_estimators': int(params["n_estimators"]),
         'learning_rate': params["learning_rate"],
         'subsample': 0.8,
         'eta': 0.1,
         'disable_default_eval_metric' : 1} 
         #'tree_method' : 'gpu_hist'}
         #'gpu_id' : 1}
    
    cv_result = xgb.cv(params_, dtrain, num_boost_round=30, feval=evalerror, nfold=3)
    
    return -cv_result['test-r2-mean'].iloc[-1]

In [7]:
for subject in subject_id:

#if subject == '000': 
#    continue
#subject = '005'
    with open('dat\\sub_'+subject+'_patient_concat.json') as f:
        data = json.load(f)
        channels = list(data.keys())

        d_xgb = {}

        for ch in channels:
            #if "STN_RIGHT" not in ch:
            #    continue
            X = np.array(data[ch]['data']).T
            y_ = np.array(data[ch]['true_movements'])
            mov_ch = data[ch]['mov_ch']

            d_xgb[ch] = {}

            for y_idx in range(y_.shape[0]):
                print('subject: '+str(subject)+' channel: '+str(ch)+' mov: '+str(mov_ch[y_idx]))
                y = y_[y_idx,:]

                X_time, y_time = append_time_dim(X[:,[4,6]], y, 5)

                dtrain = xgb.DMatrix(X_time, label=y_time)

                #reg = GradientBoostingRegressor()
                res_gp = gp_minimize(objective_XGB, space_XGB, n_calls=10, random_state=0)

                d_xgb[ch][mov_ch[y_idx]] = res_gp
    with open('bo_xgb_betagamma_sub_'+subject+'.p', 'wb') as handle:
        pickle.dump(d_xgb, handle, protocol=pickle.HIGHEST_PROTOCOL)


subject: 000 channel: ECOG_RIGHT_0 mov: MOV_RIGHT
{'max_depth': 60, 'learning_rate': 0.16646722614124754, 'n_estimators': 858, 'gamma': 8.62526564905713}
{'max_depth': 63, 'learning_rate': 0.0008354270566129826, 'n_estimators': 298, 'gamma': 1.5104167958569887}
{'max_depth': 28, 'learning_rate': 0.0024452630570839917, 'n_estimators': 812, 'gamma': 5.319794551375517}
{'max_depth': 40, 'learning_rate': 0.15149343663386153, 'n_estimators': 338, 'gamma': 6.8335468484607755}
{'max_depth': 37, 'learning_rate': 0.6106267064873843, 'n_estimators': 141, 'gamma': 8.830785325225929}
{'max_depth': 48, 'learning_rate': 0.10105405843110161, 'n_estimators': 521, 'gamma': 7.109915771070644}
{'max_depth': 72, 'learning_rate': 0.008130157526376589, 'n_estimators': 538, 'gamma': 7.827540618901216}
{'max_depth': 11, 'learning_rate': 0.002333469328026273, 'n_estimators': 187, 'gamma': 7.632263594160624}
{'max_depth': 22, 'learning_rate': 4.743412208473003e-05, 'n_estimators': 325, 'gamma': 2.34707380465314

{'max_depth': 37, 'learning_rate': 0.6106267064873843, 'n_estimators': 141, 'gamma': 8.830785325225929}
{'max_depth': 48, 'learning_rate': 0.10105405843110161, 'n_estimators': 521, 'gamma': 7.109915771070644}
{'max_depth': 72, 'learning_rate': 0.008130157526376589, 'n_estimators': 538, 'gamma': 7.827540618901216}
{'max_depth': 11, 'learning_rate': 0.002333469328026273, 'n_estimators': 187, 'gamma': 7.632263594160624}
{'max_depth': 22, 'learning_rate': 4.743412208473003e-05, 'n_estimators': 325, 'gamma': 2.3470738046531485}
{'max_depth': 23, 'learning_rate': 0.0008559431203360228, 'n_estimators': 903, 'gamma': 5.049549909201049}
subject: 000 channel: ECOG_RIGHT_4 mov: MOV_RIGHT
{'max_depth': 60, 'learning_rate': 0.16646722614124754, 'n_estimators': 858, 'gamma': 8.62526564905713}
{'max_depth': 63, 'learning_rate': 0.0008354270566129826, 'n_estimators': 298, 'gamma': 1.5104167958569887}
{'max_depth': 28, 'learning_rate': 0.0024452630570839917, 'n_estimators': 812, 'gamma': 5.319794551375

{'max_depth': 23, 'learning_rate': 0.0008559431203360228, 'n_estimators': 903, 'gamma': 5.049549909201049}
subject: 000 channel: STN_RIGHT_1 mov: MOV_LEFT
{'max_depth': 60, 'learning_rate': 0.16646722614124754, 'n_estimators': 858, 'gamma': 8.62526564905713}
{'max_depth': 63, 'learning_rate': 0.0008354270566129826, 'n_estimators': 298, 'gamma': 1.5104167958569887}
{'max_depth': 28, 'learning_rate': 0.0024452630570839917, 'n_estimators': 812, 'gamma': 5.319794551375517}
{'max_depth': 40, 'learning_rate': 0.15149343663386153, 'n_estimators': 338, 'gamma': 6.8335468484607755}
{'max_depth': 37, 'learning_rate': 0.6106267064873843, 'n_estimators': 141, 'gamma': 8.830785325225929}
{'max_depth': 48, 'learning_rate': 0.10105405843110161, 'n_estimators': 521, 'gamma': 7.109915771070644}
{'max_depth': 72, 'learning_rate': 0.008130157526376589, 'n_estimators': 538, 'gamma': 7.827540618901216}
{'max_depth': 11, 'learning_rate': 0.002333469328026273, 'n_estimators': 187, 'gamma': 7.632263594160624}

{'max_depth': 40, 'learning_rate': 0.15149343663386153, 'n_estimators': 338, 'gamma': 6.8335468484607755}
{'max_depth': 37, 'learning_rate': 0.6106267064873843, 'n_estimators': 141, 'gamma': 8.830785325225929}
{'max_depth': 48, 'learning_rate': 0.10105405843110161, 'n_estimators': 521, 'gamma': 7.109915771070644}
{'max_depth': 72, 'learning_rate': 0.008130157526376589, 'n_estimators': 538, 'gamma': 7.827540618901216}
{'max_depth': 11, 'learning_rate': 0.002333469328026273, 'n_estimators': 187, 'gamma': 7.632263594160624}
{'max_depth': 22, 'learning_rate': 4.743412208473003e-05, 'n_estimators': 325, 'gamma': 2.3470738046531485}
{'max_depth': 23, 'learning_rate': 0.0008559431203360228, 'n_estimators': 903, 'gamma': 5.049549909201049}
subject: 001 channel: ECOG_RIGHT_1 mov: MOV_LEFT
{'max_depth': 60, 'learning_rate': 0.16646722614124754, 'n_estimators': 858, 'gamma': 8.62526564905713}
{'max_depth': 63, 'learning_rate': 0.0008354270566129826, 'n_estimators': 298, 'gamma': 1.510416795856988

{'max_depth': 11, 'learning_rate': 0.002333469328026273, 'n_estimators': 187, 'gamma': 7.632263594160624}
{'max_depth': 22, 'learning_rate': 4.743412208473003e-05, 'n_estimators': 325, 'gamma': 2.3470738046531485}
{'max_depth': 23, 'learning_rate': 0.0008559431203360228, 'n_estimators': 903, 'gamma': 5.049549909201049}
subject: 001 channel: ECOG_RIGHT_5 mov: MOV_RIGHT
{'max_depth': 60, 'learning_rate': 0.16646722614124754, 'n_estimators': 858, 'gamma': 8.62526564905713}
{'max_depth': 63, 'learning_rate': 0.0008354270566129826, 'n_estimators': 298, 'gamma': 1.5104167958569887}
{'max_depth': 28, 'learning_rate': 0.0024452630570839917, 'n_estimators': 812, 'gamma': 5.319794551375517}
{'max_depth': 40, 'learning_rate': 0.15149343663386153, 'n_estimators': 338, 'gamma': 6.8335468484607755}
{'max_depth': 37, 'learning_rate': 0.6106267064873843, 'n_estimators': 141, 'gamma': 8.830785325225929}
{'max_depth': 48, 'learning_rate': 0.10105405843110161, 'n_estimators': 521, 'gamma': 7.109915771070

...

{'max_depth': 11, 'learning_rate': 0.002333469328026273, 'n_estimators': 187, 'gamma': 7.632263594160624}
{'max_depth': 22, 'learning_rate': 4.743412208473003e-05, 'n_estimators': 325, 'gamma': 2.3470738046531485}
{'max_depth': 23, 'learning_rate': 0.0008559431203360228, 'n_estimators': 903, 'gamma': 5.049549909201049}
subject: 013 channel: ECOG_LEFT_2 mov: MOV_LEFT
{'max_depth': 60, 'learning_rate': 0.16646722614124754, 'n_estimators': 858, 'gamma': 8.62526564905713}
{'max_depth': 63, 'learning_rate': 0.0008354270566129826, 'n_estimators': 298, 'gamma': 1.5104167958569887}
{'max_depth': 28, 'learning_rate': 0.0024452630570839917, 'n_estimators': 812, 'gamma': 5.319794551375517}
{'max_depth': 40, 'learning_rate': 0.15149343663386153, 'n_estimators': 338, 'gamma': 6.8335468484607755}
{'max_depth': 37, 'learning_rate': 0.6106267064873843, 'n_estimators': 141, 'gamma': 8.830785325225929}
{'max_depth': 48, 'learning_rate': 0.10105405843110161, 'n_estimators': 521, 'gamma': 7.10991577107064

{'max_depth': 63, 'learning_rate': 0.0008354270566129826, 'n_estimators': 298, 'gamma': 1.5104167958569887}
{'max_depth': 28, 'learning_rate': 0.0024452630570839917, 'n_estimators': 812, 'gamma': 5.319794551375517}
{'max_depth': 40, 'learning_rate': 0.15149343663386153, 'n_estimators': 338, 'gamma': 6.8335468484607755}
{'max_depth': 37, 'learning_rate': 0.6106267064873843, 'n_estimators': 141, 'gamma': 8.830785325225929}
{'max_depth': 48, 'learning_rate': 0.10105405843110161, 'n_estimators': 521, 'gamma': 7.109915771070644}
{'max_depth': 72, 'learning_rate': 0.008130157526376589, 'n_estimators': 538, 'gamma': 7.827540618901216}
{'max_depth': 11, 'learning_rate': 0.002333469328026273, 'n_estimators': 187, 'gamma': 7.632263594160624}
{'max_depth': 22, 'learning_rate': 4.743412208473003e-05, 'n_estimators': 325, 'gamma': 2.3470738046531485}
{'max_depth': 23, 'learning_rate': 0.0008559431203360228, 'n_estimators': 903, 'gamma': 5.049549909201049}
subject: 013 channel: STN_LEFT_0 mov: MOV_L

{'max_depth': 72, 'learning_rate': 0.008130157526376589, 'n_estimators': 538, 'gamma': 7.827540618901216}
{'max_depth': 11, 'learning_rate': 0.002333469328026273, 'n_estimators': 187, 'gamma': 7.632263594160624}
{'max_depth': 22, 'learning_rate': 4.743412208473003e-05, 'n_estimators': 325, 'gamma': 2.3470738046531485}
{'max_depth': 23, 'learning_rate': 0.0008559431203360228, 'n_estimators': 903, 'gamma': 5.049549909201049}
subject: 014 channel: ECOG_RIGHT_0 mov: MOV_RIGHT
{'max_depth': 60, 'learning_rate': 0.16646722614124754, 'n_estimators': 858, 'gamma': 8.62526564905713}
{'max_depth': 63, 'learning_rate': 0.0008354270566129826, 'n_estimators': 298, 'gamma': 1.5104167958569887}
{'max_depth': 28, 'learning_rate': 0.0024452630570839917, 'n_estimators': 812, 'gamma': 5.319794551375517}
{'max_depth': 40, 'learning_rate': 0.15149343663386153, 'n_estimators': 338, 'gamma': 6.8335468484607755}
{'max_depth': 37, 'learning_rate': 0.6106267064873843, 'n_estimators': 141, 'gamma': 8.83078532522

subject: 014 channel: ECOG_RIGHT_3 mov: MOV_LEFT
{'max_depth': 60, 'learning_rate': 0.16646722614124754, 'n_estimators': 858, 'gamma': 8.62526564905713}
{'max_depth': 63, 'learning_rate': 0.0008354270566129826, 'n_estimators': 298, 'gamma': 1.5104167958569887}
{'max_depth': 28, 'learning_rate': 0.0024452630570839917, 'n_estimators': 812, 'gamma': 5.319794551375517}
{'max_depth': 40, 'learning_rate': 0.15149343663386153, 'n_estimators': 338, 'gamma': 6.8335468484607755}
{'max_depth': 37, 'learning_rate': 0.6106267064873843, 'n_estimators': 141, 'gamma': 8.830785325225929}
{'max_depth': 48, 'learning_rate': 0.10105405843110161, 'n_estimators': 521, 'gamma': 7.109915771070644}
{'max_depth': 72, 'learning_rate': 0.008130157526376589, 'n_estimators': 538, 'gamma': 7.827540618901216}
{'max_depth': 11, 'learning_rate': 0.002333469328026273, 'n_estimators': 187, 'gamma': 7.632263594160624}
{'max_depth': 22, 'learning_rate': 4.743412208473003e-05, 'n_estimators': 325, 'gamma': 2.347073804653148

{'max_depth': 37, 'learning_rate': 0.6106267064873843, 'n_estimators': 141, 'gamma': 8.830785325225929}
{'max_depth': 48, 'learning_rate': 0.10105405843110161, 'n_estimators': 521, 'gamma': 7.109915771070644}
{'max_depth': 72, 'learning_rate': 0.008130157526376589, 'n_estimators': 538, 'gamma': 7.827540618901216}
{'max_depth': 11, 'learning_rate': 0.002333469328026273, 'n_estimators': 187, 'gamma': 7.632263594160624}
{'max_depth': 22, 'learning_rate': 4.743412208473003e-05, 'n_estimators': 325, 'gamma': 2.3470738046531485}
{'max_depth': 23, 'learning_rate': 0.0008559431203360228, 'n_estimators': 903, 'gamma': 5.049549909201049}
subject: 014 channel: STN_RIGHT_1 mov: MOV_LEFT
{'max_depth': 60, 'learning_rate': 0.16646722614124754, 'n_estimators': 858, 'gamma': 8.62526564905713}
{'max_depth': 63, 'learning_rate': 0.0008354270566129826, 'n_estimators': 298, 'gamma': 1.5104167958569887}
{'max_depth': 28, 'learning_rate': 0.0024452630570839917, 'n_estimators': 812, 'gamma': 5.31979455137551